Action2	"汽车投诉信息采集：
数据源：http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml
投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态
可以采用Python爬虫，或者第三方可视化工具"			
1、完成代码（20points）
2、结果正确（20points）

In [ ]:
# 使用request + BeautifulSoup提取12365auto投诉信息
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os    #获取当前工作路径

In [ ]:
#请求URL
#request_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
# 根据request_url得到soup
def get_page_content(request_url):
    # 得到页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html=requests.get(request_url,headers=headers,timeout=10)
    content = html.text
     # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

In [ ]:
#通过BS对象来提取当前页面的内容（方法一）
def analysis(soup):
    #找出投诉框
    temp=soup.find('div',class_='tslb_b')
    #投诉编号	投诉品牌	投诉车系	投诉车型	问题简述	典型问题	投诉时间	投诉状态
    df=pandas.DataFrame(columns=['id_','brand','car_model','type_','desc','problem','datetime','status'])#定义一个空的df
    tr_list=temp.find_all('tr')
    for tr in tr_list:
        td_list=tr.find_all('td')
        df_tr=pd.DataFrame()
        if len(td_list)>0:
            df_tr = df_tr.assign(
                id_=[td_list[0].text],
                brand=[td_list[1].text],
                car_model=[td_list[2].text],
                type_=[td_list[3].text],
                desc=[td_list[4].text],
                problem=[td_list[5].text],
                datetime=[td_list[6].text],
                status =[td_list[7].text]
            )
            df=pd.concat([df,df_tr],axis=0,ignore_index=True)
    return df 

request_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
bs = get_page_content(request_url)
df =analysis(bs) 
print(df)

In [ ]:
#通过BS对象来提取当前页面的内容（方法二）
def analysis(soup):
    #找出投诉框
    temp=soup.find('div',class_='tslb_b')
    #投诉编号	投诉品牌	投诉车系	投诉车型	问题简述	典型问题	投诉时间	投诉状态
    df=pandas.DataFrame(columns=['id','brand','car_model','type','desc','problem','datetime','status'])#定义一个空的df
    tr_list=temp.find_all('tr')
    for tr in tr_list:
        temp={}
        td_list=tr.find_all('td')
        #第一个TR没有td,其余八个都有
        for td in td_list:
            if len(td_list) > 0:
                id,brand,car_model,type,desc,problem,datetime,status = \
                td_list[0].text,td_list[1].text,td_list[2].text,td_list[3].text,td_list[4].text, \
                td_list[5].text,td_list[6].text,td_list[7].text
                
                temp['id'],temp['brand'],temp['car_model'],temp['type'],temp['desc'],temp['problem'], \
                temp['datetime'],temp['status'] = \
                id,brand,car_model,type,desc,problem,datetime,status
                
                df = df.append(temp, ignore_index=True)
    return df 

request_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
bs = get_page_content(request_url)
df =analysis(bs) 
print(df)

In [ ]:
page_num = 20
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'

result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
for i in range(page_num):
    request_url = base_url+str(i+1)+'.shtml'
    soup = get_page_content(request_url)
    print(request_url)
    df = analysis(soup)
    print(df)
    result = result.append(df)
result.to_csv(r'C:\Users\Administrator\BI\calss1-2\car_complain.csv', index=False,encoding="utf_8_sig") #数据写入，index=False表示不加索引